# Herraminetas en Python

## Lock

Comenzamos con los candados de exclusión mutua. Dentro del modulo multiprocessing tenemos `Lock()`
Este cuenta con dos métodos:
- `Lock.acquire()`: Para asegurar la exclusión mutua en una sección crítica
  - Se le pueden pasar argumentos como `blocking` o `timeout`
- `Lock.release()`: Para liberar el candado una vez terminada la sección crítica.

Los procesos que comparten sección crítica necesitan pasar como un argumento al candado de su función objetivo. La función estándar para utilizar un candado es como sigue:
```python
# ccreación del candado
lock = multiprocessing.Lock()
# adquisición del candado (aquire)
with lock:
    # sección crítica
```
Esto asegura que se adquiere el candado y muestra de forma inmediata la sección crítica. También asegura que se libere incluso si hubo un error dentro de la sección crítica.

El objeto tiene otros métodos como `locked()` para verificar si el candado está adquirido

## RLock

También se cuenta con el reentrant mutex `RLock()`, este es igual un candado de exclusión mutua, pero permite que un proceso adquiera el candado más de una vez.

## Monitor

Variable de condición de proceso, también llamado monitor.
Este es un candado combinado con una variable de condicción. Este se usa para alertar a otros procesos de alguna condición. Este se encuentra dentro de multiprocessing, se utiliza como `condition = multiprocessing.Condition()`.
Se le puede pasar un candado como argumento para su constructor, pero no es recomendable `condition = multiprocessing.Condition(lock=my_lock)`

También se puede adquirir y liberar, pero la mejor opción es un con un manejador de contexto, como:
```python
...
# acquire the condition
with condition:
    # wait to be notified
    condition.wait(timeout=10)
```
La función wait esperará hasta que sea notificado, o que llegue a su tiempo maximo si indicado con timeout.
También tenemos `wait_for()` que permite esperar a que una función regrese un valor booleano.
```python
...
# acquire the condition
with condition:
    # wait to be notified and a function to return true
    condition.wait_for(all_data_collected)
```

Además, podemos notificar a otros procesos del estado de nuestro monitor con `notify()`, se le puede asignar un valor n que indique a cuantos notificar, o notificar a todos los procesos en espera con `notify_all()`.

Como ejemplo tenemos:

In [1]:
from time import sleep
from multiprocessing import Process
from multiprocessing import Condition
 
# target function to prepare some work
def task(condition):
    # block for a moment
    sleep(1)
    # notify a waiting process that the work is done
    print('Child process sending notification...', flush=True)
    with condition:
        condition.notify()
    # do something else...
    sleep(1)
 
# entry point
if __name__ == '__main__':
    # create a condition
    condition = Condition()
    # wait to be notified that the data is ready
    print('Main process waiting for data...')
    with condition:
        # start a new process to perform some work
        worker = Process(target=task, args=(condition,))
        worker.start()
        # wait to be notified
        condition.wait()
    # we know the data is ready
    print('Main process all done')

Main process waiting for data...
Child process sending notification...
Main process all done


## Semáforo

Este permite un límite en el número de procesos o hilos que pueden adquirir un candado para la sección crítica. Es una extensión del candado de exclusión mutua que permite contar los procesos que pueden asquirir el candado antes de que procesos adicionales sean bloqueados.

Tenemos la clase dentro de multiprocessing donde su constructor toma el límite del contador como un argumento, es decir `semaforo = multiprocessing.Semaphore(100)` tendríamos un semáforo con capacidad de 100 subprocesos. Así como en el candado este puede ser adquirido y liberado con los mismos métodos `acquire()` y `release()` respectivamente.


In [8]:
from multiprocessing import Semaphore

[metodo for metodo in dir(Semaphore()) if metodo.startswith('__') is False]

['_cleanup',
 '_make_methods',
 '_make_name',
 '_rand',
 '_semlock',
 'acquire',
 'get_value',
 'release']

In [9]:
class my_class():
  def __init__(self):
    self.a = 5

dir(my_class)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']